# Actualizacion de Tablas SQL
## Utilizando las nuevas tablas desde el modulo de sistematizacion

Antes que todo, es necesario que la carpeta `db/CSV/` se encuentra con los archivos nuevos -i.e., copiar y reemplazar en la carpeta las tablas nuevas.

En particular, las nuevas tablas contienen procesada la columna `macro`. Esta columna asigna una etiqueta o categoria, permitiendo agrupar distintas respuestas. 

In [1]:
import pandas as pd
import numpy as np
import os 


from use_cases.pairs import create_pair_token
from use_cases.utils.textools import clean_alt_list

from use_cases.contributions import create_table_contributions, to_sql as con_to_sql
from use_cases.emotions import create_table_emotions, to_sql as emo_to_sql 
from use_cases.personal_needs import create_table_personal_needs, to_sql as per_need_to_sql
from use_cases.country_needs import create_table_country_needs, to_sql as count_need_to_sql

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Emociones y Sentimientos

In [2]:
emotions_table = pd.read_csv('./db/CSV/emotions.csv', low_memory=False)

In [3]:
emotions_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
49416,49416,ENC_U_4554311910197589077,NaN,inconformidad,['inconformidad'],Tristeza,usa el que no se hacer nada con respecto a los...,"['usa', 'hacer', 'respecto', 'robos', 'saqueos']",True


las listas se guardan como `strings` en el csv, es necesario volver a dejarlos como `listas`

In [4]:
emotions_table['exp_tokens'] = emotions_table['exp_tokens'].apply(clean_alt_list)
emotions_table = emotions_table.replace(np.nan, '')

Debemos recalcular los pares de palabras en caso de que algo haya cambiado

In [5]:
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

guardamos/reemplazamos la nueva tabla de pares 

In [6]:
emotion_pair.to_csv('./db/CSV/emotions_pairs.csv', index=False)

y finalmente los nuevos `.sql`

In [7]:
emo_to_sql(emotions_table, './db/SQL/emotions.sql')

### Contribuciones

En el caso de contribuciones solo necesitamos guardar el nuevo `csv` como `sql`

In [8]:
contributions_table = pd.read_csv('./db/CSV/contributions.csv', low_memory=False)

In [9]:
contributions_table.sample(1)

,Unnamed: 0,id,diag_id,ind_id,text,tokens,macro,is_online
55696,55696,68289,NaN,3063273,ir a votar,"['ir', 'votar']",Participacion,False


In [10]:
contributions_table['tokens'] = contributions_table['tokens'].apply(clean_alt_list)
contributions_table = contributions_table.replace(np.nan, '')

In [11]:
con_to_sql(contributions_table, './db/SQL/contribution.sql')